In [11]:
library(ggplot2)
library(dplyr)
library(ggrepel)
library(purrr)
library(readr)
library(stringr)
library(tidyr)

In [2]:
Ath_TF_list <- read.csv('./scan_motif/Ath_TF_list.txt', sep = '\t') %>% select(Gene_ID, Family) %>% distinct

In [3]:
Ath_TF_list

Gene_ID,Family
<chr>,<chr>
AT3G25730,RAV
AT1G68840,RAV
AT1G13260,RAV
AT1G25560,RAV
AT1G50680,RAV
AT1G51120,RAV
AT1G01010,NAC
AT1G01260,bHLH
AT1G01720,NAC


In [4]:
Ath_TF_list_duplicated <- Ath_TF_list %>% filter( Gene_ID %in% 
        (Ath_TF_list %>% group_by(Gene_ID) %>% summarize(n = n()) %>% arrange(desc(n)) %>% filter(n > 1)  %>% pull(Gene_ID))
                       ) %>% arrange(Gene_ID)
Ath_TF_list_duplicated

Gene_ID,Family
<chr>,<chr>
AT1G08810,MYB
AT1G08810,MYB_related
AT2G24790,DBB
AT2G24790,CO-like
AT2G47890,DBB
AT2G47890,CO-like
AT3G23250,MYB
AT3G23250,MYB_related
AT3G46130,MYB


In [5]:
Ath_TF_list <- Ath_TF_list %>% filter( ! Gene_ID %in% Ath_TF_list_duplicated$Gene_ID)  %>% rbind(Ath_TF_list_duplicated %>% filter(!Family %in% c('MYB_related', 'CO-like', 'AP2')))

In [6]:
source('../gene_table_and_other_scripts/theme_Publication.R')

Registering fonts with R

aakar already registered with pdfFont().

Abyssinica SIL already registered with pdfFont().

Andale Mono already registered with pdfFont().

Ani already registered with pdfFont().

AnjaliOldLipi already registered with pdfFont().

Arial already registered with pdfFont().

Arial Black already registered with pdfFont().

Chandas already registered with pdfFont().

Comic Sans MS already registered with pdfFont().

Courier New already registered with pdfFont().

DejaVu Sans already registered with pdfFont().

DejaVu Sans Mono already registered with pdfFont().

DejaVu Serif already registered with pdfFont().

Dhurjati already registered with pdfFont().

Droid Sans Fallback already registered with pdfFont().

Dyuthi already registered with pdfFont().

FreeMono already registered with pdfFont().

FreeSans already registered with pdfFont().

FreeSerif already registered with pdfFont().

Gargi already registered with pdfFont().

Garuda already registered with pdfFont(

In [7]:
map_id_name <- read.csv('../gene_table_and_other_scripts/bioMartR_Ath_all_genes_info_googled.csv')
map_id_name <- map_id_name %>% dplyr::rename('gene_name' = 'external_gene_name', 'gene_id' = 'ensembl_gene_id')
sample_n(map_id_name, 2)

gene_id,gene_name
<chr>,<chr>
AT4G30360,CNGC17
AT3G19930,STP4


In [12]:
# Define the path to the directory containing the subdirectories
base_path <- './WGCNA_ATAC_integration/ame_results_with_shuffle/'
# base_path <- './WGCNA_ATAC_integration/ame_results_with_control/'

# List all subdirectories in the base_path
sub_dirs <- list.dirs(path = base_path, full.names = TRUE, recursive = FALSE)

read_and_prepare <- function(dir_path) {
  file_path <- file.path(dir_path, 'ame.tsv')
  
  # Check if the TSV file exists
  if (file.exists(file_path)) {
    data <- read_tsv(file_path, col_types = cols()) 
    
    # Check if data is a data frame and if it's not empty
    if (is.data.frame(data) && nrow(data) > 0) {
      # Check if the 'rank' column exists
      if ("rank" %in% names(data)) {
        # Proceed with filtering and mutating
        data <- data %>% 
          filter(str_detect(rank, '^\\d')) %>% 
          mutate(module = basename(dir_path))
        return(data)
      } else {
        # If 'rank' column doesn't exist, return a message or handle accordingly
        warning(paste("The 'rank' column does not exist in the file:", file_path))
        return(tibble())
      }
    } else {
      # Return an empty tibble if the data frame is empty or not a data frame
      return(tibble())
    }
  } else {
    # File does not exist, return an empty tibble
    return(tibble())
  }
}


# Apply the function to each subdirectory and combine the results
ame_result <- map_dfr(sub_dirs, read_and_prepare) %>% mutate(motif_DB = motif_DB %>% str_remove('/data/scan_motif/Ath_TF_binding_motifs_') %>% str_remove('.meme'))
# ame_result
# combined_data now contains all the data combined from each subdirectory
ame_result <- ame_result %>% left_join(map_id_name, by = c('motif_ID' = 'gene_id')) %>%
    mutate(motif_DB = motif_DB %>% str_remove('/data/scan_motif/Ath_TF_binding_motifs_')) %>%
    arrange(`adj_p-value`) %>% select(motif_ID, motif_alt_ID, consensus, `adj_p-value`, module ) %>% 
    dplyr::rename( 'enriched_module' = module) 

Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or mo

In [13]:
read_and_prepare_sequences <- function(dir_path) {
  file_path <- file.path(dir_path, 'sequences.tsv')
  
  # Check if the TSV file exists
  if (file.exists(file_path)) {
    data <- read_tsv(file_path, col_types = cols())
    
    # Check if data is a data frame and if it's not empty
    if (is.data.frame(data) && nrow(data) > 0) {
      # Perform any required filtering and mutation
      # For example, you might want to mutate the motif_DB column as you did before
      data <- data %>% 
        mutate(motif_DB = str_remove(motif_DB, '/data/scan_motif/Ath_TF_binding_motifs_'),
               enriched_module = basename(dir_path))
      return(data)
    } else {
      # Return an empty tibble if the data frame is empty or not a data frame
      return(tibble())
    }
  } else {
    # File does not exist, return an empty tibble
    return(tibble())
  }
}

# Apply the function to each subdirectory and combine the results
sequences_result <- map_dfr(sub_dirs, read_and_prepare_sequences)

Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or mo

In [14]:
sequences_result_ori <- sequences_result

In [16]:
Res_DM1_DM2d <- readRDS('../3DM_shoot/Rds_Rdata/Res_3DM.2023-04-21 14:05:06rds') %>% filter(contrast %>% str_detect('DM1/DM2d')) %>% select(AGI, log2FoldChange)

## TF-GO using AME enrichment results

In [23]:
sequences_result_l2f <- sequences_result_ori %>% mutate(TF_traget = seq_ID %>% str_extract('^AT\\dG\\d{5}')) %>% 
    filter(motif_ID != 'NA', class == 'tp') %>% 
    as.data.frame() %>% dplyr::rename(TF = motif_ID) %>% left_join(Res_DM1_DM2d, by = c('TF' = 'AGI' )) %>% 
    dplyr::rename( TF_log2FoldChange = log2FoldChange )  %>% filter(TF_log2FoldChange %>% abs > 2) %>% 
    left_join(Res_DM1_DM2d, by = c('TF_traget' = 'AGI' )) %>% 
    dplyr::rename(target_log2FoldChange = log2FoldChange) %>% 
    filter(target_log2FoldChange %>% abs > 1) 
sequences_result_l2f%>% nrow

[1] 127114

In [22]:
sequences_result_l2f_distinct <- sequences_result_l2f %>% dplyr::select(TF,TF_log2FoldChange,  TF_traget, enriched_module) %>% distinct()  %>% mutate(TF_module = paste(TF, enriched_module, sep = '_'))
sequences_result_l2f_distinct

TF,TF_log2FoldChange,TF_traget,enriched_module,TF_module
<chr>,<dbl>,<chr>,<chr>,<chr>
AT1G01060,2.039695,AT5G06690,MEblue,AT1G01060_MEblue
AT1G01060,2.039695,AT1G17200,MEblue,AT1G01060_MEblue
AT1G01060,2.039695,AT4G04330,MEblue,AT1G01060_MEblue
AT1G01060,2.039695,AT4G14740,MEblue,AT1G01060_MEblue
AT1G01060,2.039695,AT5G67390,MEblue,AT1G01060_MEblue
AT1G01060,2.039695,AT5G47330,MEblue,AT1G01060_MEblue
AT1G01060,2.039695,AT1G14440,MEblue,AT1G01060_MEblue
AT1G01060,2.039695,AT4G26530,MEblue,AT1G01060_MEblue
AT1G01060,2.039695,AT2G42540,MEblue,AT1G01060_MEblue


In [21]:
library(gprofiler2)

In [24]:
# Split the data frame into a list of named sublists by module
module_lists <- split(sequences_result_l2f_distinct$TF_traget  , sequences_result_l2f_distinct$TF_module)
module_lists %>% str#

List of 598
 $ AT1G01060_MEblue        : chr [1:115] "AT5G06690" "AT1G17200" "AT4G04330" "AT4G14740" ...
 $ AT1G01060_MEcyan        : chr [1:64] "AT1G71000" "AT1G21310" "AT1G58340" "AT5G26920" ...
 $ AT1G01060_MEdarkgreen   : chr [1:4] "AT1G21740" "AT5G54510" "AT1G60890" "AT3G12700"
 $ AT1G01060_MEtan         : chr "AT5G50450"
 $ AT1G01250_MEblue        : chr [1:188] "AT3G21190" "AT4G12730" "AT5G03760" "AT4G29020" ...
 $ AT1G01250_MEcyan        : chr [1:95] "AT1G30700" "AT3G09940" "AT5G48560" "AT2G43150" ...
 $ AT1G01250_MEpurple      : chr [1:7] "AT1G23740" "AT4G16140" "AT1G66130" "AT1G58270" ...
 $ AT1G02230_MEcyan        : chr [1:95] "AT1G67520" "AT2G15830" "AT1G53625" "AT3G47540" ...
 $ AT1G02250_MEblue        : chr [1:26] "AT5G02180" "AT3G42800" "AT5G13630" "AT1G04520" ...
 $ AT1G02250_MEcyan        : chr [1:113] "AT3G26470" "AT1G53625" "AT1G13480" "AT1G06148" ...
 $ AT1G06160_MEblue        : chr [1:90] "AT3G51740" "AT2G43010" "AT5G55570" "AT3G03265" ...
 $ AT1G06160_MEcyan       

In [ ]:
GOs = gost( module_lists,
      organism = "athaliana", sources = c("GO:BP"), multi_query = FALSE, evcodes = TRUE)

In [30]:
GOs$result ##the query is TF

query,significant,p_value,term_size,query_size,intersection_size,precision,recall,term_id,source,term_name,effective_domain_size,source_order,parents,evidence_codes,intersection
<chr>,<lgl>,<dbl>,<int>,<int>,<int>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<int>,<int>,<list>,<chr>,<chr>
AT1G01060_MEblue,TRUE,2.551912e-03,281,93,9,0.09677419,0.032028470,GO:0015979,GO:BP,photosynthesis,21899,5119,GO:0044237,"IEA,IMP,IEA,IEA,IMP,IEA,IMP,TAS IEA,IDA IEA","AT4G04330,AT3G15095,AT4G27440,AT4G12800,AT4G04640,AT1G29930,AT1G75690,AT1G18730,AT3G61470"
AT1G01060_MEcyan,TRUE,1.647722e-10,3714,48,31,0.64583333,0.008346796,GO:0006950,GO:BP,response to stress,21899,2613,GO:0050896,"IEP,IMP IGI IEP IEA,IEP,IMP IEP,IMP IEP HEP,IEA,IMP IEP,IMP,IEA,IEP HEP,IEP,IEP IEA,IMP IEP IEA,IMP,IEP IEA,IMP IEP,IMP,IEP,IMP,IMP IEP HEP,IMP,IEP IEA,IEP,IMP IEP ISS TAS NAS IEA,IEA,IEA,IMP IEP,IEP,IDA IMP,IEP,IEP TAS IEA","AT1G58340,AT5G26920,AT4G19230,AT5G39610,AT4G12720,AT3G28510,AT3G13610,AT3G04070,AT4G17500,AT5G62520,AT4G28140,AT5G40010,AT2G26560,AT1G65690,AT1G80820,AT4G23210,AT2G32150,AT4G17670,AT1G74360,AT5G59820,AT4G39830,AT1G29290,AT5G13180,AT3G52430,AT5G46520,AT4G21850,AT3G13790,AT3G61890,AT1G68820,AT5G40780,AT2G14610"
AT1G01060_MEcyan,TRUE,3.983424e-08,5967,48,35,0.72916667,0.005865594,GO:0050896,GO:BP,response to stimulus,21899,13774,GO:0008150,"IEP HEP IEA,IDA IMP IGI IEP IEA,IEP,HEP,IMP IEP,IMP IEP,IMP IEP HEP,IEA,IMP IEP,IMP,IEP IEA,IEP HEP,IEP IEA,IEP IEA,IMP IEP IEA,IMP IEP IEA,IEP IEA,IMP IEP,IEP,IMP,IEP,IMP,IMP IEP HEP,IMP,IEP IEA,IEP,IDA IMP IGI IEP ISS TAS NAS IEA,IEA,IEA,IMP IEP,IMP IEP,IDA IMP,IEA,IEP,IEP TAS IEA","AT1G58340,AT5G26920,AT4G19230,AT5G27350,AT1G61800,AT5G39610,AT4G12720,AT3G28510,AT3G13610,AT3G04070,AT4G17500,AT5G62520,AT4G28140,AT5G40010,AT2G26560,AT1G65690,AT1G80820,AT4G23210,AT4G37990,AT2G32150,AT4G17670,AT1G74360,AT5G59820,AT4G39830,AT1G29290,AT5G13180,AT3G52430,AT5G46520,AT4G21850,AT3G13790,AT3G61890,AT1G68820,AT4G35380,AT5G40780,AT2G14610"
AT1G01060_MEcyan,TRUE,5.554921e-07,2125,48,21,0.43750000,0.009882353,GO:0009628,GO:BP,response to abiotic stimulus,21899,3586,GO:0050896,"IEP,IDA IMP IEP,IEP,IMP,IEP,IEP HEP,IMP,IEP HEP,IEP,IEP,IEP,IMP,IEP,IMP,IMP IEP HEP,IEP IEA,IEP,IMP,IEP,IDA IMP,IEP","AT1G58340,AT5G26920,AT4G19230,AT1G61800,AT5G39610,AT4G12720,AT3G04070,AT5G62520,AT4G28140,AT5G40010,AT2G26560,AT1G65690,AT1G80820,AT2G32150,AT5G59820,AT1G29290,AT5G13180,AT3G52430,AT3G61890,AT1G68820,AT2G14610"
AT1G01060_MEcyan,TRUE,6.172633e-07,1910,48,20,0.41666667,0.010471204,GO:0009605,GO:BP,response to external stimulus,21899,3570,GO:0050896,"HEP,IDA IMP IGI IEP IEA,IEP,HEP,IEP,IMP,IMP,IMP IEP,IEP,IMP IEP,IMP IEP,IEP,IEP,IMP,IMP,IEA,IDA IMP IEP ISS TAS NAS IEA,IEP,IEP,IEP","AT1G58340,AT5G26920,AT4G19230,AT5G27350,AT1G61800,AT5G39610,AT4G12720,AT3G13610,AT4G17500,AT2G26560,AT4G23210,AT4G37990,AT4G17670,AT1G74360,AT4G39830,AT1G29290,AT3G52430,AT3G13790,AT3G61890,AT2G14610"
AT1G01060_MEcyan,TRUE,3.520199e-06,1455,48,17,0.35416667,0.011683849,GO:0051707,GO:BP,response to other organism,21899,14359,"GO:0043207, GO:0044419","HEP,IDA IMP IGI IEP IEA,IEP,HEP,IEP,IMP,IMP,IEP,IMP IEP,IMP IEP,IEP,IMP,IMP,IDA IMP IEP ISS TAS NAS IEA,IEP,IEP,IEP","AT1G58340,AT5G26920,AT4G19230,AT5G27350,AT1G61800,AT5G39610,AT4G12720,AT4G17500,AT2G26560,AT4G23210,AT4G37990,AT1G74360,AT4G39830,AT3G52430,AT3G13790,AT3G61890,AT2G14610"
AT1G01060_MEcyan,TRUE,3.520199e-06,1455,48,17,0.35416667,0.011683849,GO:0043207,GO:BP,response to external biotic stimulus,21899,10929,"GO:0009605, GO:0009607","HEP,IDA IMP IGI IEP IEA,IEP,HEP,IEP,IMP,IMP,IEP,IMP IEP,IMP IEP,IEP,IMP,IMP,IDA IMP IEP ISS TAS NAS IEA,IEP,IEP,IEP","AT1G58340,AT5G26920,AT4G19230,AT5G27350,AT1G61800,AT5G39610,AT4G12720,AT4G17500,AT2G26560,AT4G23210,AT4G37990,AT1G74360,AT4G39830,AT3G52430,AT3G13790,AT3G61890,AT2G14610"
AT1G01060_MEcyan,TRUE,4.096807e-06,1678,48,18,0.37500000,0.010727056,GO:1901700,GO:BP,response to oxygen-containing compound,21899,22187,GO:0042221,"IEP,IMP IEA,IEP,IEP,IEP,IMP,IMP,IEP,IMP IEP IEA,IMP,I

In [29]:
GOs$result <- GOs$result %>% filter(p_value < 0.01)

In [33]:
GOs$result%>%filter( query %>% str_detect('AT4G38000_MEblue'))

query,significant,p_value,term_size,query_size,intersection_size,precision,recall,term_id,source,term_name,effective_domain_size,source_order,parents,evidence_codes,intersection
<chr>,<lgl>,<dbl>,<int>,<int>,<int>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<int>,<int>,<list>,<chr>,<chr>
AT4G38000_MEblue,TRUE,4.457571e-34,281,485,56,0.115463918,0.19928826,GO:0015979,GO:BP,photosynthesis,21899,5119,GO:0044237,"IEA,IEA,IDA IMP IEA,IMP,IMP IEA,IMP IEA,IMP,IMP,IMP IEA,IMP,IMP,IEA,IEA,TAS IEA,IMP,IDA IEA,IEA,IEA,IEA,IMP IEA,IMP,IMP NAS IEA,IEA,IEA,IMP IEA,IMP IEA,IEA,IDA IEA,IEA,IEA,TAS IEA,IDA IMP,IEA,IEA,IEA,NAS IEA,IEA,IMP IEA,IMP IEA,IEA,IMP IEA,IEA,IEA,TAS,IDA,IEA,IDA IEA,IMP,IMP IGI,IMP IEA,IMP IC IEA,IEA,IDA IEA,IDA IMP IEA,IMP,IEA","AT2G42600,AT3G56940,AT2G05100,AT1G55370,AT5G66190,AT1G03600,AT1G75690,AT1G64770,AT4G02530,AT3G15095,AT3G01500,AT1G32060,AT1G29920,AT1G18730,AT4G25910,AT3G27690,AT1G76450,AT4G18480,AT5G38430,AT5G54270,AT3G16250,AT1G15820,AT5G38420,AT5G27390,AT1G67090,AT5G38410,AT1G74470,AT3G47470,AT1G29910,AT1G30380,AT1G06680,AT1G54500,AT5G64040,AT4G04330,AT1G12900,AT1G42970,AT1G29930,AT1G67740,AT1G15980,AT3G55330,AT4G10340,AT1G31330,AT5G13630,AT4G03280,AT2G47940,AT4G12800,AT3G61470,AT4G04640,AT1G01790,AT1G55670,AT1G08380,AT1G03130,AT2G05070,AT1G55480,AT4G01050,AT4G27440"
AT4G38000_MEblue,TRUE,5.633925e-20,146,485,32,0.065979381,0.21917808,GO:0019684,GO:BP,"photosynthesis, light reaction",21899,6208,"GO:0006091, GO:0015979","IDA IMP IEA,IMP,IMP,IMP IEA,IMP,IMP,IMP IEA,IMP,IEA,TAS IEA,IMP,IDA IEA,IMP IEA,IMP,IEA,IDA IEA,IEA,TAS,IDA IMP,IEA,IMP,IMP IEA,IMP IEA,TAS,IDA,IDA IEA,IMP,IMP,IMP IC,IDA IEA,IDA,IMP","AT2G05100,AT1G55370,AT5G66190,AT1G03600,AT1G75690,AT1G64770,AT4G02530,AT3G15095,AT1G29920,AT1G18730,AT4G25910,AT3G27690,AT5G54270,AT3G16250,AT1G15820,AT3G47470,AT1G29910,AT1G06680,AT1G54500,AT1G29930,AT1G67740,AT1G15980,AT4G10340,AT4G03280,AT2G47940,AT3G61470,AT4G04640,AT1G55670,AT1G08380,AT2G05070,AT1G55480,AT4G01050"
AT4G38000_MEblue,TRUE,4.795367e-16,37,485,17,0.035051546,0.45945946,GO:0009765,GO:BP,"photosynthesis, light harvesting",21899,3700,"GO:0006091, GO:0019684","IDA IMP IEA,IMP IEA,IMP,IMP IEA,IEA,IDA IEA,IMP IEA,IEA,IDA IEA,IEA,IEA,IEA,IDA,IDA IEA,IC,IDA IEA,IDA","AT2G05100,AT1G03600,AT1G75690,AT4G02530,AT1G29920,AT3G27690,AT5G54270,AT1G15820,AT3G47470,AT1G29910,AT1G29930,AT4G10340,AT2G47940,AT3G61470,AT1G08380,AT2G05070,AT1G55480"
AT4G38000_MEblue,TRUE,4.617037e-11,465,485,41,0.084536082,0.08817204,GO:0006091,GO:BP,generation of precursor metabolites and energy,21899,1974,GO:0044237,"IEA,IDA IMP IEA,IMP,IMP,ISS IEA,IMP IEA,IMP,IMP,IMP IEA,IMP,IEA,IEA,TAS IEA,IMP,IDA IEA,IEA,IMP IEA,IMP IEA,IEA,ISS IEA,IDA IEA,IEA,TAS,IDA IMP,IEA,IEA,IMP,IMP IEA,IMP IEA,ISS IEA,TAS,IDA,IDA IEA,IMP,IMP,IMP IC,IEA,IDA IEA,IDA,IMP,IEA","AT2G42600,AT2G05100,AT1G55370,AT5G66190,AT2G21330,AT1G03600,AT1G75690,AT1G64770,AT4G02530,AT3G15095,AT1G64640,AT1G29920,AT1G18730,AT4G25910,AT3G27690,AT2G25060,AT5G54270,AT3G16250,AT1G15820,AT4G38970,AT3G47470,AT1G29910,AT1G06680,AT1G54500,AT3G20440,AT1G29930,AT1G67740,AT1G15980,AT4G10340,AT4G26530,AT4G03280,AT2G47940,AT3G61470,AT4G04640,AT1G55670,AT1G08380,AT1G20340,AT2G05070,AT1G55480,AT4G01050,AT1G76100"
AT4G38000_MEblue,TRUE,5.063277e-10,762,485,52,0.107216495,0.06824147,GO:0009416,GO:BP,response to light stimulus,21899,3517,GO:0009314,"IMP,IEP,IMP,IMP IEP,IEP,IEP,IDA IMP IEP,IMP TAS,IMP,IMP IGI IEA,IEP,IMP,IMP,IMP IEP,IEP,IMP IEP IEA,IMP IEP IEA,IMP,IDA IMP,IDA IEP,IEP,IMP,IEP,IEP,IEP IEA,IPI IMP,IEP,IEP,IMP,IEP,IMP,IMP,IEP,IEP,IEP,IEP,IEP,IEP,IEP,IMP,IMP,IDA,IMP,IEP,IEP,IMP,IGI,IDA IEP,IMP,IEP,IDA IMP IGI IEA,IMP IEA","AT5G23280,AT1G15550,AT1G18400,AT5G56860,AT4G14130,AT5G04140,AT2G05100,AT5G63310,AT1G73870,AT3G45780,AT1G14440,AT1G26945,AT1G03600,AT5G49330,AT4G37930,AT2G36990,AT2G34640,AT1G10522,AT5G07690,AT3G27690,AT5G49730,AT5G15160,AT1G69530,AT4G38860,AT5G13730,AT5G64330,AT2G45400,AT5G54270,AT3G01490,AT2G42540,AT5G39860,AT1G15820,AT2G06850,AT3G47470,AT1G68010,AT2G39730,AT1G42970,AT4G25050,AT

In [34]:
GOs$result%>%filter( query %>% str_detect('AT4G38000_MEblue'), term_name %>% str_detect("auxin")) 

query,significant,p_value,term_size,query_size,intersection_size,precision,recall,term_id,source,term_name,effective_domain_size,source_order,parents,evidence_codes,intersection
<chr>,<lgl>,<dbl>,<int>,<int>,<int>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<int>,<int>,<list>,<chr>,<chr>


In [82]:
gp_mod <- GOs$result[, c("query", "source", "term_id",
                      str_detecterm_name", "p_value", "query_size", 
                          "intersection_size", "term_size", 
                          "effective_domain_size", "intersection")]

In [83]:
gp_mod %>% filter(query == 'AT3G56400_MEcyan') 

query,source,term_id,term_name,p_value,query_size,intersection_size,term_size,effective_domain_size,intersection
<chr>,<chr>,<chr>,<chr>,<dbl>,<int>,<int>,<int>,<int>,<chr>
AT3G56400_MEcyan,GO:BP,GO:0006950,response to stress,3.033531e-09,104,48,3714,21899,"AT3G49110,AT5G25440,AT5G53120,AT4G37530,AT1G21520,AT3G52470,AT1G56060,AT5G62520,AT3G44720,AT5G64100,AT1G45145,AT4G11170,AT2G17220,AT3G53480,AT5G24090,AT2G45220,AT5G40780,AT5G46050,AT4G08780,AT2G43570,AT4G12490,AT2G45760,AT1G66090,AT1G80820,AT3G04070,AT1G21550,AT2G13810,AT1G02360,AT2G14560,AT1G33560,AT4G19520,AT3G04720,AT3G49210,AT3G11340,AT4G23210,AT1G35910,AT5G44380,AT3G61190,AT1G30720,AT1G06160,AT5G22540,AT2G47130,AT3G13610,AT3G28510,AT5G13320,AT1G14870,AT3G13790,AT5G45380"
AT3G56400_MEcyan,GO:BP,GO:0006952,defense response,4.215840e-08,104,30,1629,21899,"AT3G49110,AT5G25440,AT5G53120,AT3G52470,AT3G44720,AT1G45145,AT4G11170,AT2G17220,AT5G24090,AT2G45220,AT5G40780,AT5G46050,AT2G43570,AT4G12490,AT2G45760,AT1G66090,AT1G80820,AT2G13810,AT2G14560,AT1G33560,AT4G19520,AT3G04720,AT3G11340,AT4G23210,AT3G61190,AT1G06160,AT5G22540,AT2G47130,AT5G13320,AT3G13790"
AT3G56400_MEcyan,GO:BP,GO:0050896,response to stimulus,1.688733e-07,104,59,5967,21899,"AT3G49110,AT5G25440,AT5G53120,AT4G37530,AT1G21520,AT3G52470,AT1G56060,AT5G62520,AT3G44720,AT5G64100,AT1G45145,AT4G11170,AT2G17220,AT3G53480,AT1G74080,AT5G24090,AT2G45220,AT5G40780,AT4G04960,AT5G46050,AT4G08780,AT2G43570,AT4G12490,AT2G47000,AT2G45760,AT1G66090,AT1G80820,AT3G04070,AT1G21550,AT2G13810,AT1G02360,AT2G14560,AT1G33560,AT4G19520,AT3G04720,AT3G49210,AT3G11340,AT5G45090,AT4G23210,AT1G35910,AT5G62480,AT5G44380,AT5G50760,AT3G61190,AT1G69930,AT1G30720,AT1G06160,AT5G22540,AT5G27350,AT2G47130,AT3G13610,AT3G28510,AT5G13320,AT1G14870,AT4G08950,AT5G48400,AT1G01560,AT3G13790,AT5G45380"
AT3G56400_MEcyan,GO:BP,GO:0009605,response to external stimulus,3.393141e-05,104,28,1910,21899,"AT3G49110,AT5G25440,AT5G53120,AT3G52470,AT1G45145,AT3G53480,AT1G74080,AT5G24090,AT2G45220,AT5G46050,AT2G43570,AT4G12490,AT2G47000,AT2G13810,AT1G02360,AT2G14560,AT1G33560,AT3G04720,AT3G11340,AT4G23210,AT1G06160,AT5G22540,AT5G27350,AT2G47130,AT3G13610,AT5G13320,AT3G13790,AT5G45380"
AT3G56400_MEcyan,GO:BP,GO:0051707,response to other organism,4.500968e-05,104,24,1455,21899,"AT3G49110,AT5G25440,AT5G53120,AT3G52470,AT1G45145,AT1G74080,AT5G24090,AT2G45220,AT5G46050,AT2G43570,AT4G12490,AT2G13810,AT1G02360,AT2G14560,AT1G33560,AT3G04720,AT3G11340,AT4G23210,AT1G06160,AT5G22540,AT5G27350,AT2G47130,AT5G13320,AT3G13790"
AT3G56400_MEcyan,GO:BP,GO:0043207,response to external biotic stimulus,4.500968e-05,104,24,1455,21899,"AT3G49110,AT5G25440,AT5G53120,AT3G52470,AT1G45145,AT1G74080,AT5G24090,AT2G45220,AT5G46050,AT2G43570,AT4G12490,AT2G13810,AT1G02360,AT2G14560,AT1G33560,AT3G04720,AT3G11340,AT4G23210,AT1G06160,AT5G22540,AT5G27350,AT2G47130,AT5G13320,AT3G13790"
AT3G56400_MEcyan,GO:BP,GO:0044419,biological process involved in interspecies interaction between organisms,5.814119e-05,104,24,1475,21899,"AT3G49110,AT5G25440,AT5G53120,AT3G52470,AT1G45145,AT1G74080,AT5G24090,AT2G45220,AT5G46050,AT2G43570,AT4G12490,AT2G13810,AT1G02360,AT2G14560,AT1G33560,AT3G04720,AT3G11340,AT4G23210,AT1G06160,AT5G22540,AT5G27350,AT2G47130,AT5G13320,AT3G13790"
AT3G56400_MEcyan,GO:BP,GO:0009607,response to biotic stimulus,6.352256e-05,104,24,1482,21899,"AT3G49110,AT5G25440,AT5G53120,AT3G52470,AT1G45145,AT1G74080,AT5G24090,AT2G45220,AT5G46050,AT2G43570,AT4G12490,AT2G13810,AT1G02360,AT2G14560,AT1G33560,AT3G04720,AT3G11340,AT4G23210,AT1G06160,AT5G22540,AT5G27350,AT2G47130,AT5G13320,AT3G13790"
AT3G56400_MEcyan,GO:BP,GO:0042742,defense response to bacterium,1.178774e-04,104,12,363,21899,"AT3G49110,AT5G25440,AT5G53120,AT2G45220,AT5G46050,AT2G13810,AT1G33560,AT3G04720,AT4G23210,AT5G22540,AT2G47130,AT5G13320"


In [36]:
sequences_result_l2f_distinct %>% filter(enriched_module == 'MEblue')

TF,TF_log2FoldChange,TF_traget,enriched_module,TF_module
<chr>,<dbl>,<chr>,<chr>,<chr>
AT1G01060,2.039695,AT5G06690,MEblue,AT1G01060_MEblue
AT1G01060,2.039695,AT1G17200,MEblue,AT1G01060_MEblue
AT1G01060,2.039695,AT4G04330,MEblue,AT1G01060_MEblue
AT1G01060,2.039695,AT4G14740,MEblue,AT1G01060_MEblue
AT1G01060,2.039695,AT5G67390,MEblue,AT1G01060_MEblue
AT1G01060,2.039695,AT5G47330,MEblue,AT1G01060_MEblue
AT1G01060,2.039695,AT1G14440,MEblue,AT1G01060_MEblue
AT1G01060,2.039695,AT4G26530,MEblue,AT1G01060_MEblue
AT1G01060,2.039695,AT2G42540,MEblue,AT1G01060_MEblue
